# ML Assignment 2 & 3   
# Paul Thillen, Louis-Philippe Noël
# 3. PRACTICAL PART (40 pts) : Neural network implementation and experiments

## Numerically stable softmax. 
You will need to compute a numerically
stable softmax. Refer to posted readings to see how to do this. Start by
writing the expression for a single vector, then adapt it for a mini-batch
of examples stored in a matrix.


In [36]:
%pylab inline
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import scipy.io
import gzip,pickle
import copy

Populating the interactive namespace from numpy and matplotlib


#Load data

In [37]:
#two moons 
two_moons = np.loadtxt(open('2moons.txt','r'))
#mnist
f=gzip.open('mnist.pkl.gz')
mnist=pickle.load(f)

#Utilities

In [38]:
def softmax(x):
    maximum = np.max(x, axis=-1, keepdims=True)
    top = np.exp(x - maximum)
    bottom = np.sum(top, axis=-1, keepdims=True)
    return top/bottom

#test
#A= array([[ 1.,  3.,  4.,  5.,  6.],
#       [ 4.,  5.,  6.,  3.,  3.],
#       [ 1.,  3.,  4.,  5.,  6.],
#       [ 4.,  5.,  6.,  3.,  3.]])
#softmax(A)

In [39]:
def uniforme(a,b):
    return(a+(b-a)*np.random.rand()) 

## Parameter initialization. 
As you know, it is necessary to randomly
initialize the parameters of your neural network (trying to avoid symme-
try and saturating neurons, and ideally so that the pre-activation lies in
the bending region of the activation function so that the overall networks
acts as a non linear function). We suggest that you sample the weights
of a layer from a uniform distribution in [ -1/sqrt(n_c), 1/sqrt(n_c) ], 
where n c is the number of inputs for this layer (changing from one layer to the other).
Biases can be initialized at 0. Justify any other initialization method.

## fprop
fprop will compute the forward progpagation i.e. step by step computation from the input to the output and the cost of the activations of each layer.

## bprop
bprop will use the computed activations by fprop and does
the backpropagation of the gradients from the cost to the input following
precisely the steps derived in part 2.

## Finite difference gradient check. We can estimate the gradient nu-
merically using the finite difference method. You will implement this estimate as a tool to check your gradient computation. To do so, calculate the value of the loss function for the current parameter values (for a single example or a mini batch). Then for each scalar parameter θ k , change the
parameter value by adding a small perturbation (10^−6 < ε < 10^−4 )
and calculate the new value of the loss (same example or minibatch),
then set the value of the parameter back to its original value. The partial
derivative with respect to this parameter is estimated by dividing the
change in the loss function by ε. The ratio of your gradient computed
by backpropagation and your estimate using finite difference should be
between 0.99 and 1.01.

All in one : Class

In [40]:
class Model:

    def plot_function(self, train_data, title):
        plt.figure()
        d1 = train_data[train_data[:, -1] > 0]
        d2 = train_data[train_data[:, -1] == 0]
        plt.scatter(d1[:, 0], d1[:, 1], c='b', label='classe 1')
        plt.scatter(d2[:, 0], d2[:, 1], c='g', label='classe 0')
        x = np.linspace(np.min(train_data[:, 0]) - 0.5,
                        np.max(train_data[:, 0]) + 0.5,
                        100)
        y = -(self.weights[0]*x + self.bias - .5)/self.weights[1]
        plt.plot(x, y, c='r', lw=2, label='y = -(w1*x + b1)/w2')
        plt.xlim(np.min(train_data[:, 0]) - 0.5, np.max(train_data[:, 0]) + 0.5)
        plt.ylim(np.min(train_data[:, 1]) - 0.5, np.max(train_data[:, 1]) + 0.5)
        plt.grid()
        plt.legend(loc='lower right')
        plt.title(title)
        plt.show()

In [43]:
class neural_net:
    
    def __init__(self, nc, m, k):
        #nc : number of neurons in the hidden layer
        #m : number of classes
        #k : size of mini-batches to use
        self.nc = nc
        self.m = m
        self.k = k
        
        
    def ini(self,train_data):
        self.train_data = train_data
        self.x = np.transpose(train_data[:,:-1])
        self.y = train_data[:,-1]
        self.b1=np.zeros(self.nc)
        self.b2=np.zeros(self.m)
        self.nc1 = self.x.shape[1]
        self.nc2 = self.nc
        self.w1 = np.multiply(np.ones((self.nc,self.nc1)),uniforme(-1/sqrt(self.nc1),1/sqrt(self.nc1)))
        self.w2 = np.multiply(np.ones((self.m,self.nc)),uniforme(-1/sqrt(self.nc2),1/sqrt(self.nc2)))
        
    def fprop(self):
        print x
        #ha : activation des neurones de la couche cachée
        self.ha= np.add(np.dot(self.w1,np.transpose(self.x)),self.b1)
        #hs : sortie des neurones de la couche cachée
        self.hs= np.maximum(0,self.ha)
        #oa : activation des neurones de la couche de sortie
        self.oa= np.add(np.dot(self.w2,self.hs),self.b2)
        #os : sortie des neurones de la couche cachée
        self.os=softmax(self.oa)
        #L : fonction de coût
        self.L= -np.log(self.os)
        print(self.L)
        return self.L
    
    def bprop(self):
        #grad_oa : gradient de la fonction d'activation de la couche de sortie par rapport à L
        onehot=np.zeros(self.m)
        onehot[np.int_(self.y)-1]=1
        self.grad_oa = self.os-onehot
        #grad_w2 et grad_b2
        self.grad_w2 = np.dot(self.grad_oa,np.transpose(self.hs))
        self.grad_b2 = self.grad_oa
        #grad_hs
        self.grad_hs = np.dot(np.transpose(self.w2),self.grad_oa)
        #grad_ha
        def I_ha(x):
            #x is a vector
            y=np.zeros(len(x))
            for i in range(0, len(x)):
                    if x[0,i]>0:
                        y[i] = 1
                    else:
                        y[i]=0
            return y

        self.grad_ha = np.multiply(self.grad_hs,I_ha(self.ha))
        #grad_w1 et grad_b1
        self.grad_w1 = np.dot(self.grad_ha,self.x)
        self.grad_b1 = self.grad_ha
        #grad_x
        self.grad_x = np.dot(np.transpose(self.w1),self.grad_ha)
        #elastic regularization
        self.grad_w2_el = self.grad_w2+2*self.w2-np.sign(self.w2)
        self.grad_w1_el = self.grad_w1+2*self.w1-np.sign(self.w1)
        #to remove
        print("self.grad_x=",self.grad_x)
        #print("I_ha(x)=",I_ha(self.ha))
        #print("self.grad_ha=",self.grad_ha)
        #print("self.grad_w1=",self.grad_w1)
        
    #check if slightly changed parameters yield a similar gradient
    def finite_check(self):
        e = 0.00002
        #copier le réseau actuel et 
        net_copy = copy.copy(self)
        net_copy.w1 = np.add(self.w1,e)
        net_copy.w2 = np.add(self.w2,e)
        net_copy.b1 = np.add(self.b1,e)
        net_copy.b2 = np.add(self.b2,e)
        #gradients avec nouvelles valeurs des paramètres
        net_copy.fprop()
        net_copy.bprop()
        #check difference
        ratio_w1 = np.divide(self.grad_w1,net_copy.grad_w1)
        print ratio_w1
        ratio_w2 = np.divide(self.grad_w2,net_copy.grad_w2)
        print ratio_w2
        ratio_b1 = np.divide(self.grad_b1,net_copy.grad_b1)
        print ratio_b1
        ratio_b2 = np.divide(self.grad_b2,net_copy.grad_b2)
        print ratio_b2
        ratios = [ratio_w1,ratio_w2,ratio_b1,ratio_b2]
        #sortie
        for ratio in ratios:
            if (ratio<0.99).any() or (ratio>1.01).any():
                return False
        return True

 ## Size of the mini batches. 
We ask that your computation and gradient descent is done on minibatches (as opposed to the whole training set)
with adjustable size using a hyperparameter K. In the minibatch case,
we do not manipulate a single input vector, but rather a batch of input
vectors grouped in a matrix (that will give a matrix representation at
each layer, and for the input). In the case where the size is one, we obtain
an equivalent to the stochastic gradient. Given that numpy is eﬃcient on
matrix operations, it is more eﬃcient to perform computations on a whole
minibatch. It will greatly impact the execution time.

1. As a beginning, start with an implementation that computes the gradients
for a single example, and check that the gradient is correct using the ﬁnite
diﬀerence method described above.

In [54]:
#exemple x : 2 neurones cachés, 2 dimensions, classe = 1
x= np.array([[-1.2084724, 0.39429077, 1.]])
net1 = neural_net(2,2,1)
net1.ini(x)
net1.fprop()
net1.bprop()
net1.finite_check()

[[-1.2084724   0.39429077  1.        ]]
[[ 0.59648971  0.80015685]
 [ 0.59648971  0.80015685]]
('self.grad_x=', array([[ 0.        , -0.46412037]]))
[[-1.2084724   0.39429077  1.        ]]
[[ 0.59648335  0.80016465]
 [ 0.59648335  0.80016465]]
('self.grad_x=', array([[ 0.        , -0.46411025]]))
[[ 1.00005479]
 [ 1.00005479]]
[[ 0.99989125  0.99989125]
 [ 0.99989125  0.99989125]]
[[        nan  1.00005479]
 [        nan  1.00005479]]
[[ 1.0000078  1.0000078]
 [ 1.0000078  1.0000078]]


/Users/louis-philippenoel/anaconda/envs/python27/lib/python2.7/site-packages/ipykernel_launcher.py:91: RuntimeWarning: invalid value encountered in divide
/Users/louis-philippenoel/anaconda/envs/python27/lib/python2.7/site-packages/ipykernel_launcher.py:98: RuntimeWarning: invalid value encountered in less
/Users/louis-philippenoel/anaconda/envs/python27/lib/python2.7/site-packages/ipykernel_launcher.py:98: RuntimeWarning: invalid value encountered in greater


True

-2. Display the gradients for both methods (direct computation and ﬁnite
difference) for a small network (e.g. d = 2 and d h = 2) with random
weights and for a single example.

-3. Add a hyperparameter for the minibatch size K to allow compute the
gradients on a minibatch of K examples (in a matrix), by looping over
the K examples (this is a small addition to your previous code).

-4. Display the gradients for both methods (direct computation and finite
difference) for a small network (e.g. d = 2 and dh = 2) with random
weights and for a minibatch with 10 examples (you can use examples from
both classes from the dataset 2 moons).

-5. Train your neural network using gradient descent on the dataset of the
two moons. Plot the decision regions for several different values of the
hyperparameters (weight decay, number of hidden units, early stopping)
so as to illustrate their effect on the capacity of the model.

-6. As a second step, copy your existing implementation to modify it to a new
implementation that will use matrix calculus (instead of a loop) on batches
of size K to improve efficiency. Take the matrix expressions in numpy
derived in the first part, and adapt them for a minibatch of size
K. Show in your report what you have modified (precise the
former and new expressions with the shapes of each matrices).

-7. Compare both implementations (with a loop and with matrix calculus)
to check that they both give the same values for the gradients on the
parameters, first for K = 1, then for K = 10. Display the gradients for
both methods.

-8. Time how long takes an epoch on MNIST (1 epoch = 1 full traversal
through the whole training set) for K = 100 for both versions (loop over
a minibatch and matrix calculus).

-9. Adapt your code to compute the error (proportion of misclassified examples)
on the training set as well as the total loss on the training set during each
epoch of the training procedure, and at the end of each epoch, it computes
the error and average loss on the validation set and the test set. Display
the 6 corresponding figures (error and average loss on train/valid/test),
and write them in a log file.

-10. Train your network on the MNIST dataset. Plot the training/valid/test
curves (error and loss as a function of the epoch number, corresponding
to what you wrote in a file in the last question). Include in your report
the curves obtained using your best hyperparameters, i.e. for which you
obtained your best error on the validation set. We suggest 2 plots : the
first one will plot the error rate (train/valid/test with different colors,
precise which color in a legend) and the other one for the averaged loss
(on train/valid/test). You should be able to get less than 5% test error.
Indicate the values of your best hyperparameters corresponding to the
curves. Bonus points are given for a test error of less that 2%.